In [1]:
from dataclasses import dataclass

@dataclass
class Base:
    val: int = 0

@dataclass
class Derived(Base):
    val: int = 2

In [2]:
A = Derived()
print(A.val)

2


In [3]:
import jax.numpy as jnp
a = jnp.array([0])
a.shape

(1,)